In [ ]:
!pip install transformers
!pip install seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving fake.csv to fake.csv
Saving true.csv to true.csv


In [ ]:
import nltk

In [ ]:
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

file_fake = pd.read_csv("fake.csv", encoding = "ISO-8859-1")
file_real = pd.read_csv("true.csv", encoding = "ISO-8859-1")

file_fake = file_fake.drop(["subject", "date"], axis = 1)
file_real = file_real.drop(["subject", "date"], axis = 1)
file_fake["label"] = "fake news"
file_real["label"] = "real news"

final_dataset = pd.concat([file_fake, file_real])

nltk.download('stopwords')
stop = set(stopwords.words('english'))

def preprocess(text):
    clean_text = " "
    text = text.lower()
    clean_text = re.sub(r'[^\w\s]', ' ', text)
    clean_t = clean_text.split()
    filtered_text = []
    for i in clean_t:
        if i not in stop:
            filtered_text.append(i)
    clean_text = " ".join(filtered_text)
    return clean_text

def tokenization(text):
    clean = preprocess(text)
    token = clean.split(" ")
    count = Counter(token)
    return token, count

final_dataset["text_token"], final_dataset["text_token_count"] = zip(*final_dataset["text"].apply(tokenization))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def text_to_bert_embeddings(text_list, tokenizer, model, max_length=128, device='cuda'):
    model.to(device)
    embeddings = []
    for text in text_list:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoded['input_ids'].to(device)
        attention_mask = encoded['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            cls_embedding = outputs.last_hidden_state[:,0,:].squeeze().cpu().numpy()
        embeddings.append(cls_embedding)
    return embeddings


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

X = final_dataset["text"].tolist()
y = final_dataset["label"].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

X_train_embeddings = text_to_bert_embeddings(X_train, tokenizer, bert_model)
X_test_embeddings = text_to_bert_embeddings(X_test, tokenizer, bert_model)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_embeddings, y_train)

predictions = clf.predict(X_test_embeddings)
acc = accuracy_score(y_test, predictions)
print("BERT Feature Extraction Accuracy:", acc)
print("Classification Report:\n", classification_report(y_test, predictions))


BERT Feature Extraction Accuracy: 0.9986636971046771
Classification Report:
               precision    recall  f1-score   support

   fake news       1.00      1.00      1.00      5876
   real news       1.00      1.00      1.00      5349

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

